<a href="https://colab.research.google.com/github/JuyeongNam/HAI-Hecto_AI_Challenge_2025_DL_Smartcity_final/blob/main/pilot_JNam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import torchvision.transforms as transforms
import timm

In [2]:
CFG = {
    'Img_size': 224,
    'Batch_size': 8,
    'EPOCHS': 3,
    'Learning_rate': 1e-4,
    'Weight_decay': 1e-4,
    'SEED': 42,
    'Num_class' : 396
}

In [3]:
range(CFG['Num_class']), range(396)

(range(0, 396), range(0, 396))

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# 1. CustomImageDataset 클래스 정의
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None, is_test=False):
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if self.is_test:
            # Test: 이미지 파일 경로만 저장 (레이블 없음)
            for fname in sorted(os.listdir(root_dir)):
                if fname.lower().endswith('.jpg'):
                    img_path = os.path.join(root_dir, fname)
                    self.samples.append((img_path,))
        else:
            # Train/Validation: 폴더 구조에서 클래스별 레이블 추출
            self.classes = sorted(os.listdir(root_dir))
            self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
            for cls_name in self.classes:
                cls_folder = os.path.join(root_dir, cls_name)
                for fname in os.listdir(cls_folder):
                    if fname.lower().endswith('.jpg'):
                        img_path = os.path.join(cls_folder, fname)
                        label = self.class_to_idx[cls_name]
                        self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if self.is_test:
            img_path = self.samples[idx][0]
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, os.path.basename(img_path)  # (image, filename) 반환
        else:
            img_path, label = self.samples[idx]
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label

# 2. 데이터 전처리(Transforms) 정의
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((CFG['Img_size'], CFG['Img_size']), scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((CFG['Img_size'], CFG['Img_size'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((CFG['Img_size'], CFG['Img_size'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [6]:
# 3. 데이터셋 로딩 및 학습/검증 셋 분리
train_root = '/content/drive/MyDrive/DL_Smartcity_final/sample_data/train'
full_dataset = CustomImageDataset(train_root, transform=None, is_test=False)
print(f"총 이미지 수 (전체): {len(full_dataset)}")

# 레이블만 추출하여 Stratified Split 진행
targets = [label for _, label in full_dataset.samples]
class_names = full_dataset.classes

train_idx, val_idx = train_test_split(
    np.arange(len(targets)),
    test_size=0.2,
    stratify=targets,
    random_state=CFG['SEED']
)

# Subset을 활용하여 Transform을 적용한 Dataset 생성
train_dataset = Subset(CustomImageDataset(train_root, transform=train_transform, is_test=False), train_idx)
val_dataset = Subset(CustomImageDataset(train_root, transform=val_transform, is_test=False), val_idx)

print(f"Train 이미지 개수: {len(train_dataset)}, Valid 이미지 개수: {len(val_dataset)}")

총 이미지 수 (전체): 3106
Train 이미지 개수: 2484, Valid 이미지 개수: 622


In [7]:
# 4. DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=CFG['Batch_size'], shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=CFG['Batch_size'], shuffle=False, num_workers=0)

# 5. 테스트용 DataLoader 정의
test_root = '/content/drive/MyDrive/DL_Smartcity_final/sample_data/test'
test_dataset = CustomImageDataset(test_root, transform=val_transform, is_test=True)
test_loader  = DataLoader(test_dataset, batch_size=CFG['Batch_size'], shuffle=False, num_workers=0)

In [8]:
val_dataset = Subset(CustomImageDataset(train_root, transform=test_transform, is_test=False), val_idx)
val_loader  = DataLoader(val_dataset, batch_size=CFG['Batch_size'], shuffle=False, num_workers=2)

# 검증용 배치 예시
images_val, labels_val = next(iter(val_loader))
print(f"Valid Batch 이미지 크기: {images_val.shape}, 레이블 예시: {labels_val.tolist()}")

Valid Batch 이미지 크기: torch.Size([8, 3, 224, 224]), 레이블 예시: [272, 196, 388, 129, 234, 370, 34, 107]


In [9]:
class DenseNetModel(nn.Module):
    def __init__(self, num_classes):
        super(DenseNetModel, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()
        self.head = nn.Linear(in_features, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x

In [10]:
!pip install torchinfo

In [11]:
from torchinfo import summary
model = DenseNetModel(num_classes=CFG['Num_class'])
summary(model)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 129MB/s]


Layer (type:depth-idx)                        Param #
DenseNetModel                                 --
├─DenseNet: 1-1                               --
│    └─Sequential: 2-1                        --
│    │    └─Conv2d: 3-1                       9,408
│    │    └─BatchNorm2d: 3-2                  128
│    │    └─ReLU: 3-3                         --
│    │    └─MaxPool2d: 3-4                    --
│    │    └─_DenseBlock: 3-5                  335,040
│    │    └─_Transition: 3-6                  33,280
│    │    └─_DenseBlock: 3-7                  919,680
│    │    └─_Transition: 3-8                  132,096
│    │    └─_DenseBlock: 3-9                  2,837,760
│    │    └─_Transition: 3-10                 526,336
│    │    └─_DenseBlock: 3-11                 2,158,080
│    │    └─BatchNorm2d: 3-12                 2,048
│    └─Identity: 2-2                          --
├─Linear: 1-2                                 405,900
Total params: 7,359,756
Trainable params: 7,359,756
Non-trainab

In [12]:
model = model.to(device)

# (3) 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(
    model.parameters(),
    lr=CFG['Learning_rate'],
    weight_decay=CFG['Weight_decay']
)

# (4) Learning Rate 스케줄러 (선택 사항)
#    여기서는 간단히 CosineAnnealingLR을 예시로 추가했습니다.
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                T_max=CFG['EPOCHS'],
                                                eta_min=1e-6)

In [13]:
best_val_logloss = float('inf')

for epoch in range(1, CFG['EPOCHS'] + 1):
    # === 5-1. Train 단계 ===
    model.train()
    total_train_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch}/{CFG['EPOCHS']}] Train"):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)           # 로짓(logits) 크기: (B, num_classes)
        loss = criterion(outputs, labels) # CrossEntropyLoss
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # === 5-2. Validation 단계 ===
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    all_probs = []   # 검증용 확률(softmax) 저장
    all_labels = []  # 검증용 실제 라벨 저장

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"[Epoch {epoch}/{CFG['EPOCHS']}] Valid"):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

            # 정확도 계산
            _, preds = torch.max(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # LogLoss 계산을 위한 확률과 실제 라벨 저장
            probs = F.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    val_logloss = log_loss(y_true=all_labels,
                           y_pred=all_probs,
                           labels=list(range(CFG['Num_class'])))

    # 스케줄러 한 스텝(step) 업데이트
    scheduler.step()

    print(
        f"\nEpoch {epoch}/{CFG['EPOCHS']} ▶ "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Valid Loss: {avg_val_loss:.4f} | "
        f"Valid Acc: {val_accuracy:.2f}% | "
        f"Valid LogLoss: {val_logloss:.4f}"
    )

    # 검증 LogLoss 기준으로 최고 성능 모델 저장
    if val_logloss < best_val_logloss:
        best_val_logloss = val_logloss
        torch.save(model.state_dict(), 'best_densenet121.pth')
        print(f"📦 [모델 저장] Epoch {epoch} 에서 LogLoss {val_logloss:.4f} 달성\n")
    else:
        print()


[Epoch 1/3] Valid: 100%|██████████| 78/78 [04:00<00:00,  3.08s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(



Epoch 1/3 ▶ Train Loss: 6.2340 | Valid Loss: 5.9018 | Valid Acc: 1.61% | Valid LogLoss: 5.9024
📦 [모델 저장] Epoch 1 에서 LogLoss 5.9024 달성



[Epoch 2/3] Valid: 100%|██████████| 78/78 [00:05<00:00, 13.89it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(



Epoch 2/3 ▶ Train Loss: 5.6559 | Valid Loss: 5.4829 | Valid Acc: 6.27% | Valid LogLoss: 5.4830
📦 [모델 저장] Epoch 2 에서 LogLoss 5.4830 달성



[Epoch 3/3] Valid: 100%|██████████| 78/78 [00:05<00:00, 14.11it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(



Epoch 3/3 ▶ Train Loss: 5.1528 | Valid Loss: 5.3190 | Valid Acc: 8.68% | Valid LogLoss: 5.3190
📦 [모델 저장] Epoch 3 에서 LogLoss 5.3190 달성

